In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Flatten, Dropout, GlobalAveragePooling1D, SimpleRNN, GRU, LSTM, Reshape
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.callbacks import EarlyStopping

# Load and preprocess the data
X_train = np.loadtxt("../Datasets/CAPP Dataset/SubjectIndependent50PercentOverlap/X_train.txt")
y_train = np.loadtxt("../Datasets/CAPP Dataset/SubjectIndependent50PercentOverlap/y_train.txt")
X_test = np.loadtxt("../Datasets/CAPP Dataset/SubjectIndependent50PercentOverlap/X_test.txt")
y_test = np.loadtxt("../Datasets/CAPP Dataset/SubjectIndependent50PercentOverlap/y_test.txt")

n_timesteps = 100
n_features = 9
X_train = X_train.reshape(X_train.shape[0], n_timesteps, n_features)
X_test = X_test.reshape(X_test.shape[0], n_timesteps, n_features)

lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)

n_outputs = y_train.shape[1]

2024-09-27 05:45:16.921220: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-27 05:45:16.932207: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-27 05:45:16.935525: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-27 05:45:16.944585: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-27 05:45:17.490909: W tensorflow/compiler/tf2

In [2]:
# Define model architectures
def create_1d_cnn(n_timesteps, n_features, n_outputs):
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps, n_features)),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        GlobalAveragePooling1D(),
        Dense(50, activation='relu'),
        Dense(n_outputs, activation='softmax')
    ])
    return model

def create_simple_rnn(n_timesteps, n_features, n_outputs):
    model = Sequential([
        SimpleRNN(50, return_sequences=True, input_shape=(n_timesteps, n_features)),
        SimpleRNN(50),
        Dense(30, activation='relu'),
        Dense(n_outputs, activation='softmax')
    ])
    return model

def create_gru(n_timesteps, n_features, n_outputs):
    model = Sequential([
        GRU(50, return_sequences=True, input_shape=(n_timesteps, n_features)),
        GRU(50),
        Dense(30, activation='relu'),
        Dense(n_outputs, activation='softmax')
    ])
    return model

def create_lightweight_lstm(n_timesteps, n_features, n_outputs):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(n_timesteps, n_features)),
        LSTM(50),
        Dense(30, activation='relu'),
        Dense(n_outputs, activation='softmax')
    ])
    return model

def create_cnn_gru(n_timesteps, n_features, n_outputs):
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps, n_features)),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        GRU(50, return_sequences=True),
        GRU(50),
        Dense(n_outputs, activation='softmax')
    ])
    return model

# Function to train and evaluate a model
def train_and_evaluate(model, model_name):
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    history = model.fit(
        X_train, y_train,
        epochs=100,
        batch_size=2048,
        validation_split=0.2,
        callbacks=[early_stopping],
        verbose=1
    )

    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)

    accuracy = accuracy_score(y_true_classes, y_pred_classes)
    print(f"\n{model_name} Accuracy: {accuracy:.5f}")
    print("\nClassification Report:")
    print(classification_report(y_true_classes, y_pred_classes, digits=5))

    return model, accuracy

# Train and evaluate models
models = [
    ("1D CNN", create_1d_cnn(n_timesteps, n_features, n_outputs)),
    ("Simple RNN", create_simple_rnn(n_timesteps, n_features, n_outputs)),
    ("GRU", create_gru(n_timesteps, n_features, n_outputs)),
    ("Lightweight LSTM", create_lightweight_lstm(n_timesteps, n_features, n_outputs)),
    ("1D CNN + GRU", create_cnn_gru(n_timesteps, n_features, n_outputs))
]

results = []

for model_name, model in models:
    print(f"\nTraining {model_name}...")
    trained_model, accuracy = train_and_evaluate(model, model_name)
    results.append((model_name, trained_model, accuracy))

# Print summary of results
print("\nModel Performance Summary:")
for model_name, _, accuracy in results:
    print(f"{model_name}: {accuracy:.5f}")

# Identify the best performing model
best_model = max(results, key=lambda x: x[2])
print(f"\nBest performing model: {best_model[0]} with accuracy {best_model[2]:.5f}")

# Save the best model
best_model[1].save(f'best_model_{best_model[0].replace(" ", "_").lower()}.h5')
print(f"Best model saved as 'best_model_{best_model[0].replace(' ', '_').lower()}.h5'")

/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1727394339.679596   58666 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727394339.711294   58666 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:


Training 1D CNN...
Epoch 1/100


I0000 00:00:1727394341.190097   59007 service.cc:146] XLA service 0x7263b800ade0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727394341.190124   59007 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2024-09-27 05:45:41.215012: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-27 05:45:41.308153: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
2024-09-27 05:45:41.691526: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_483', 20 bytes spill stores, 20 bytes spill loads

2024-09-27 05:45:42.404729: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in functio

25/39 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1117 - loss: 2.9445

I0000 00:00:1727394345.706641   59007 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - accuracy: 0.3988 - loss: 1.9342 - val_accuracy: 0.5195 - val_loss: 1.6521
Epoch 3/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - accuracy: 0.6007 - loss: 1.3348 - val_accuracy: 0.5991 - val_loss: 1.3621
Epoch 4/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - accuracy: 0.7086 - loss: 0.9691 - val_accuracy: 0.6376 - val_loss: 1.1856
Epoch 5/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - accuracy: 0.7647 - loss: 0.7736 - val_accuracy: 0.6618 - val_loss: 1.0942
Epoch 6/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.7980 - loss: 0.6581 - val_accuracy: 0.6737 - val_loss: 1.0548
Epoch 7/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.8188 - loss: 0.5864 - val_accuracy: 0.6903 - val_loss: 1.0204
Epoch 8/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.8336 - loss: 0.5325 - val_accuracy: 0.6888 - val_loss: 1.0275
Epoch 9/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - accuracy: 0.8490 - loss: 0.4837 - val_accuracy: 0.7098 - val_l

W0000 00:00:1727394558.572005   59001 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727394558.572574   59001 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727394558.573070   59001 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727394558.573568   59001 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727394558.574073   59001 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727394558.574571   59001 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727394558.575044   59001 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727394558.575557   59001 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727394558.576045   59001 gp

616/616 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step


W0000 00:00:1727394560.656959   59006 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727394560.657533   59006 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727394560.658038   59006 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727394560.658558   59006 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727394560.659056   59006 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727394560.659563   59006 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727394560.660042   59006 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727394560.660542   59006 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727394560.661033   59006 gp


1D CNN + GRU Accuracy: 0.68751

Classification Report:
              precision    recall  f1-score   support

           0    0.79513   0.77454   0.78470       927
           1    0.99550   0.95469   0.97467       927
           2    0.78986   0.80952   0.79957       924
           3    0.90132   0.62486   0.73805       877
           4    0.72250   0.61099   0.66208       946
           5    0.54545   0.68298   0.60652       940
           6    0.77586   0.85805   0.81489       944
           7    0.71920   0.79263   0.75413       950
           8    0.60888   0.30284   0.40449       951
           9    0.67816   0.19094   0.29798       927
          10    0.87306   0.68154   0.76551      1322
          11    0.54603   0.92553   0.68684       564
          12    0.81982   0.87781   0.84783       933
          13    0.65444   0.95503   0.77667       934
          14    0.61068   0.49894   0.54918       940
          15    0.44811   0.61422   0.51818       928
          16    0.96728  